<a href="https://colab.research.google.com/github/dctec/mlc/blob/master/week2/StockPrediction/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
print(sys.version)

3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]


**Step 1** 
- Go to Yahoo Finance and find a stock price that you want to predict. Click on historical data, then download the CSV of that stocks price history https://finance.yahoo.com/quote/AAPL/history?p=AAPL  

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pandas_datareader.data as web
from pandas import Series, DataFrame

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [17]:
%pwd

'/content/drive/My Drive/mlc/week2/drive/My Drive/mlc/week2/drive/My Drive/mlc/week2'

In [18]:
%mkdir -p "/content/drive/My Drive/mlc/week2"
%cd "/content/drive/My Drive/mlc/week2"
%mkdir "sample_data"

/content/drive/My Drive/mlc/week2


In [0]:
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2019, 9, 4)

df = web.DataReader("IBM", 'yahoo', start, end)
df.tail()
df.to_csv('sample_data/IBM.csv',index=False)

In [5]:
%cd "/content/drive/My Drive/mlc/week2"
df = pd.read_csv('sample_data/IBM.csv')
df.tail()

/content/drive/My Drive/mlc/week2


,High,Low,Open,Close,Volume,Adj Close
416,132.889999,130.039993,130.440002,132.759995,2713600.0,132.759995
417,135.690002,134.039993,134.179993,134.880005,2971800.0,134.880005
418,136.070007,134.360001,135.580002,135.529999,2960600.0,135.529999
419,134.899994,133.330002,134.850006,134.100006,2827900.0,134.100006
420,136.429993,135.149994,135.710007,136.320007,2247900.0,136.320007


In [0]:
close_px = df['Adj Close']
mavg = close_px.rolling(window=100).mean()

In [6]:
df.dtypes

High         float64
Low          float64
Open         float64
Close        float64
Volume       float64
Adj Close    float64
dtype: object

In [10]:
df.tail(5)

,High,Low,Open,Close,Volume,Adj Close
416,132.889999,130.039993,130.440002,132.759995,2713600.0,132.759995
417,135.690002,134.039993,134.179993,134.880005,2971800.0,134.880005
418,136.070007,134.360001,135.580002,135.529999,2960600.0,135.529999
419,134.899994,133.330002,134.850006,134.100006,2827900.0,134.100006
420,136.429993,135.149994,135.710007,136.320007,2247900.0,136.320007


In [9]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid",color_codes=True)
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.countplot('Adj Close',data=df)

ValueError: ignored